# GenAI v3: Background Manipulation for Attention Control

**Zero training required!**

## How It Works
- **Product is AUTO-DETECTED** using SAM + DINO (no keyword mask needed!)
- **Episodic memory** tracks consistent elements across video frames
- **Background is modified** to control attention (product stays unchanged)
- `increase`: Make background less distracting → more attention on product
- `decrease`: Make background more interesting → less attention on product

## Models Used
- **SDXL Inpainting** - State-of-the-art background editing
- **SAM ViT-H** - Segment Anything for object detection
- **DINOv2** - Feature tracking across frames

In [ ]:
from GenAI_v3 import SceneManipulator

In [ ]:
# Initialize (loads SDXL + SAM + DINO, ~12GB total)
manipulator = SceneManipulator(
    valid_scenes_file="data/valid_scenes.csv",  # Optional (for scene timing)
    video_dir="data/data_tiktok",
    output_dir="outputs/genai_v3",
    device="cuda",
    auto_detect_product=True,  # Auto-detect main product (default)
)

## Increase Attention on Product

Makes background less distracting (muted, simple) → viewer focuses on product

In [ ]:
# INCREASE attention on product in scene 6
output = manipulator.manipulate(
    video_id="YOUR_VIDEO_ID",  # ← Change this
    scene_index=6,              # ← Change this
    action="increase",
)

print(f"Output: {output}")

## Decrease Attention on Product

Makes background more interesting (vibrant, detailed) → viewer distracted from product

In [ ]:
# DECREASE attention on product in scene 3
output = manipulator.manipulate(
    video_id="YOUR_VIDEO_ID",  # ← Change this
    scene_index=3,              # ← Change this
    action="decrease",
)

print(f"Output: {output}")

## Adjust Strength

- `strength=0.5`: Subtle change
- `strength=0.8`: Default, noticeable change
- `strength=1.0`: Maximum change

In [ ]:
# Subtle background change
output = manipulator.manipulate(
    video_id="YOUR_VIDEO_ID",
    scene_index=6,
    action="increase",
    strength=0.5,  # ← Subtle
)

## Batch Processing

In [ ]:
import pandas as pd

# Load valid scenes
scenes_df = pd.read_csv("data/valid_scenes.csv")

# Process multiple videos
for video_id in scenes_df['video_id'].unique()[:3]:  # First 3 videos
    try:
        output = manipulator.manipulate(
            video_id=str(video_id),
            scene_index=1,  # First scene
            action="increase",
        )
        print(f"✓ {video_id} -> {output}")
    except Exception as e:
        print(f"✗ {video_id}: {e}")

## Summary

### What It Does
1. Load video frames
2. **Auto-detect main product** using SAM + DINO (episodic memory)
3. **Invert mask** → select background (not product)
4. Use SDXL to modify background only
5. Replace scene in video with smooth blending
6. Export edited video

### Why Auto-Detection?
- **Keywords can be unreliable** - CLIPSeg might miss the actual product
- **Episodic memory** - Tracks what appears consistently across frames
- **No manual labeling** - Works out of the box
- **More robust** - Finds the actual main content

### Why Background?
- **Product stays authentic** - no fake edits
- **Attention is contextual** - background affects focus
- **A/B testing** - clean comparison (same product, different context)

### Performance
- ~60s per scene (including auto-detection)
- ~12GB GPU memory peak
- No training required!